# Numerical 1-D FEM solution of the heat diffusion equation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sy
from scipy.sparse import csr_matrix
from scipy import sparse
from numpy import array

First we define some numerical constants and define the problem

In [ ]:
#constants
lx          = 10           #width
nnod        = 7            #number of nodes
nnodel      = 2            #nodes per element
k           = 1            #conductivity
Q           = 1            #source term
Tleft       = 0            #T boundary conditions
Trite       = 0

#numerics
nel         = nnod-1       #numnber of elements



Now we make our FEM mesh and element connectivity

In [ ]:
Gcoord, dx      = np.linspace(0,lx,nnod,   retstep=True) #global coordinates
EL2NOD      = np.array([np.arange(1,nnod), np.arange(2,nnod+1)])  # connectivity matrix

Now we spell out our analytically-derived element stiffness matrix (see previous sections):

In [ ]:
Ael    = np.array([[k/dx, -k/dx],[-k/dx, k/dx]])
Rhs_el  = np.array([Q*dx/2, Q*dx/2])

And allocate the global Rhs and coefficient matrix arrays

In [5]:
A_all   = np.zeros((nnod, nnod))
Rhs_all = np.zeros(nnod)

Now we need to loop over all elements and assemble the global stiffness matrix

In [7]:
I = []
J = []
K = []
At = np.array([[1,2], [3,4]])
print(At)
#J = array([1,2])
#V = array([1,1])
#A = sparse.coo_matrix((V,(I,J)),shape=(4,4))
#print(A)

I.append((EL2NOD[:,0]*np.ones((nnodel,1))).T.reshape(nnodel*nnodel))
print(I)
J.append((np.ones((2,1)) * EL2NOD[:,0]).reshape(4))
print(J)
K.append(At.reshape(4))
print(K)

[[1 2]
 [3 4]]
[array([1., 1., 2., 2.])]
[array([1., 2., 1., 2.])]
[array([1, 2, 3, 4])]


In [7]:
I = []
J = []
K = []
for iel in range(0,nel):
    
    I.append(list((EL2NOD[:,iel]*np.ones((nnodel,1))).T.reshape(nnodel*nnodel)))
    J.append(list((np.ones((nnodel,1)) * EL2NOD[:,0]).reshape(nnodel*nnodel)))
    K.append(list(Ael.reshape(nnodel*nnodel)))
       
    #A[EL2NOD[0,iel]-1,EL2NOD[0,iel]-1] = A[list(EL2NOD[:,iel]-1), list(EL2NOD[:,iel]-1)] + Ael
print(I)
print(J)
I=np.array(I,dtype=int).reshape(-1)-1
J=np.array(J,dtype=int).reshape(-1)-1
K=np.array(K,dtype=float).reshape(-1)

A = sparse.coo_matrix((K,(I,J)),shape=(nnod,nnod))
print(A)

[[1.0, 1.0, 2.0, 2.0], [2.0, 2.0, 3.0, 3.0], [3.0, 3.0, 4.0, 4.0], [4.0, 4.0, 5.0, 5.0], [5.0, 5.0, 6.0, 6.0], [6.0, 6.0, 7.0, 7.0]]
[[1.0, 2.0, 1.0, 2.0], [1.0, 2.0, 1.0, 2.0], [1.0, 2.0, 1.0, 2.0], [1.0, 2.0, 1.0, 2.0], [1.0, 2.0, 1.0, 2.0], [1.0, 2.0, 1.0, 2.0]]
  (0, 0)	0.6
  (0, 1)	-0.6
  (1, 0)	-0.6
  (1, 1)	0.6
  (1, 0)	0.6
  (1, 1)	-0.6
  (2, 0)	-0.6
  (2, 1)	0.6
  (2, 0)	0.6
  (2, 1)	-0.6
  (3, 0)	-0.6
  (3, 1)	0.6
  (3, 0)	0.6
  (3, 1)	-0.6
  (4, 0)	-0.6
  (4, 1)	0.6
  (4, 0)	0.6
  (4, 1)	-0.6
  (5, 0)	-0.6
  (5, 1)	0.6
  (5, 0)	0.6
  (5, 1)	-0.6
  (6, 0)	-0.6
  (6, 1)	0.6


In [ ]:

%local stiffness matrix and local RHS
K_el    = [k/dx, -k/dx; -k/dx, k/dx];
Rhs_el  = [Q*dx/2; Q*dx/2];

%storage global stiffness matrix and RHS vector
K_all   = sparse(nnod,nnod);
Rhs_all = zeros(nnod,1);

%assembly loop over all elements
for iel=1:nel
    
    %sort local stiffness matrix into global
    K_all(EL2NOD(iel,:), EL2NOD(iel,:)) = K_all(EL2NOD(iel,:), EL2NOD(iel,:)) + K_el;
    
    %sort local RHS into global RHS
    Rhs_all(EL2NOD(iel,:)) = Rhs_all(EL2NOD(iel,:)) + Rhs_el;
end
    
%Apply boundary conditions
K_all(1,:) = 0; K_all(1,1) = 1; Rhs_all(1) = Tleft;
K_all(end,:) = 0; K_all(end,end) = 1; Rhs_all(end) = Trite;

%and solve
T = K_all\Rhs_all;

%Analytical solution
X     = linspace(0,lx, 100*nnod);
T_ana = -1/2*Q/k*X.^2 + 1/2*Q/k*lx*X;

%Plotting
figure(1),
plot(Gcoord,T,'-o',X,T_ana);
legend('FEM', 'Analytical');
xlabel('Distance')
ylabel('Temperature');
box on
grid on



